In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets.samples_generator import make_classification

from gcforest.utils.config_utils import load_json
from gcforest.fgnet import FGNet, FGTrainConfig

X, y = make_classification(n_samples=100, n_features=784, n_classes=10, n_informative=8)
X = X.reshape(-1, 1, 28, 28)
# y = y.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2017)

config = load_json('model.json')
train_config = FGTrainConfig(config["train"])

net = FGNet(config["net"], train_config.data_cache)
net.fit_transform(X_train, y_train, X_test, y_test, train_config)

[ 2017-09-29 13:05:54,247][fgnet.fit_transform] X_train.shape=(8000L, 1L, 28L, 28L), y_train.shape=(8000L,), X_test.shape=(2000L, 1L, 28L, 28L), y_test.shape=(2000L,)
[ 2017-09-29 13:05:54,250][fg_win_layer.fit_transform] [data][win1/7x7], bottoms=[u'X', u'y'], tops=[u'win1/7x7/ets', u'win1/7x7/rf']
[ 2017-09-29 13:05:54,252][fg_win_layer.fit_transform] [progress][win1/7x7] ti=0/2, top_name=win1/7x7/ets
[ 2017-09-29 13:05:54,253][fg_win_layer.fit_transform] [data][win1/7x7,train] bottoms.shape=8000x1x28x28,8000
[ 2017-09-29 13:05:54,275][win_utils.get_windows] get_windows_start: X.shape=(8000L, 1L, 28L, 28L), X_win.shape=(49L, 968000L), nw=11, nh=11, c=1, win_x=7, win_y=7, stride_x=2, stride_y=2
[ 2017-09-29 13:05:54,494][win_utils.get_windows] get_windows_end
[ 2017-09-29 13:05:54,500][fg_win_layer.fit_transform] [data][win1/7x7,test] bottoms.shape=2000x1x28x28,2000
[ 2017-09-29 13:05:54,509][win_utils.get_windows] get_windows_start: X.shape=(2000L, 1L, 28L, 28L), X_win.shape=(49L, 24

KeyboardInterrupt: 